## 1. IMPORTS

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 2. SCALED DOT-PRODUCT ATTENTION   

`ScaledDotProductAttention` es el mecanismo de atención central utilizado por el componente de atención multicabezal para calcular las puntuaciones de atención.    

Calcula el producto punto entre las consultas y las claves, escala el resultado, aplica una máscara (si es necesario) y, a continuación, calcula la suma ponderada de los valores en función de los pesos de atención.

In [ ]:
# -----------------------------------------------------------------------------
# Clase 1: Scaled Dot-Product Attention
# -----------------------------------------------------------------------------
# Esta clase implementa el cálculo básico de la "atención" (attention).
# Recibe tres tensores: Q (query), K (key) y V (value). 
# El mecanismo:
#   - Se calcula la similitud (producto punto) entre Q y K.
#   - Se escala dividiendo entre la raíz de la dimensión (d_k).
#   - Se aplica softmax para obtener coeficientes de importancia (atención).
#   - Se multiplica esos coeficientes por V para obtener la salida.
# -----------------------------------------------------------------------------
class ScaledDotProductAttention(layers.Layer):
    def __init__(self, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)

    def call(self, q, k, v, mask=None):
        # q, k, v tienen forma (batch_size, num_heads, seq_len, depth)

        # 1) Producto punto Q*K^T para obtener la matriz de atención
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        
        # d_k = depth (dimensión de k)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        # 2) Escalado por raíz de d_k
        scaled_attention_logits = matmul_qk / tf.sqrt(dk)

        # Opcional: Se puede aplicar máscara (por ejemplo en el decodificador).
        if mask is not None:
            # Máscara se suele aplicar sumando un valor muy negativo,
            # para anular esos valores tras el softmax.
            scaled_attention_logits += (mask * -1e9)

        # 3) Softmax para convertir logits en coeficientes de atención
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        # 4) Multiplicamos los pesos de atención por V
        output = tf.matmul(attention_weights, v)  # (batch_size, num_heads, seq_len_q, depth)

        return output, attention_weights

## 3. MULTI-HEAD ATTENTION   

El mecanismo de atención multicabezal (Multi-Head Attention) permite al modelo centrarse simultáneamente en distintas partes de la secuencia de entrada.    
Utiliza varias cabezas de atención para calcular distintas representaciones de la entrada.

- `Multi-Head Attention`: Esta clase realiza la atención multicabezal dividiendo la entrada en múltiples cabezales, lo que permite al modelo centrarse en diferentes partes de la secuencia simultáneamente.
- `d_model` y `num_heads`: `d_model` es el tamaño del embedding y `num_heads` se refiere al número de cabezas de atención.
- Capas densas: Las transformaciones lineales de las consultas(queries), claves(keys) y valores(values) se crean mediante `wq`, `wk` y `wv`.
- `split_heads`: Divide el tensor de entrada en varias cabezas. El tensor resultante tendrá la forma `(batch_size, num_heads, seq_len, depth)`.
- `call`: Este método realiza la operación de atención propiamente dicha. Primero calcula las consultas(queries), claves(keys) y valores(values) aplicando las capas densas correspondientes, las divide en cabezas y, a continuación, calcula la atención `scaled_attention`.  

In [ ]:
# -----------------------------------------------------------------------------
# Clase 2: Multi-Head Attention
# -----------------------------------------------------------------------------
# El transformer divide la atención en varios "cabezas" (heads).
# Se hace una proyección lineal de Q, K y V para cada cabeza, se aplica
# atención (ScaledDotProductAttention) de forma independiente, y luego
# se concatena el resultado para devolverlo a la dimensión original.
# -----------------------------------------------------------------------------
class MultiHeadAttention(layers.Layer):
    def __init__(self, d_model, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        
        # Verificamos que d_model sea divisible por num_heads
        assert d_model % num_heads == 0

        self.depth = d_model // num_heads

        # Capas densas para proyectar Q, K y V
        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)

        # Capa final para la proyección tras la concatenación
        self.dense = layers.Dense(d_model)
        
        # Clase para la atención con producto punto escalado
        self.attention = ScaledDotProductAttention()

    def split_heads(self, x, batch_size):
        """
        Divide la última dimensión en (num_heads, depth).
        Reordena la salida a (batch_size, num_heads, seq_len, depth).
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])  # para que num_heads quede en la segunda posición

    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]

        # Proyectamos Q, K y V
        q = self.wq(q)  # (batch_size, seq_len_q, d_model)
        k = self.wk(k)  # (batch_size, seq_len_k, d_model)
        v = self.wv(v)  # (batch_size, seq_len_v, d_model)

        # Dividimos cada una en num_heads
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # Aplicamos atención escalada
        scaled_attention, attention_weights = self.attention(q, k, v, mask=mask)

        # Reordenamos de vuelta y concatenamos
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        # Proyección final
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

## 4. CAPA FEED-FORWARD    

La capa feed-forward posicional se utiliza para procesar cada posición de forma independiente:   

- `PositionwiseFeedforward`: Esta clase aplica dos capas densas a cada posición de forma independiente. La primera capa transforma la entrada a una dimensión superior y la segunda la reduce de nuevo al tamaño original de `d_model`.
- `call`: Aplica las capas feed-forward secuencialmente a la entrada.

In [ ]:

# -----------------------------------------------------------------------------
# Clase 3: Feed Forward (Positionalwise Feed-Forward Network)
# -----------------------------------------------------------------------------
# Cada "bloque" del transformer tiene una red densa de dos capas.
# La primera expande la dimensionalidad (ff_dim) y la segunda la reduce de nuevo.
# -----------------------------------------------------------------------------
class PositionwiseFeedForward(layers.Layer):
    def __init__(self, d_model, ff_dim, **kwargs):
        super(PositionwiseFeedForward, self).__init__(**kwargs)
        self.dense1 = layers.Dense(ff_dim, activation='relu')
        self.dense2 = layers.Dense(d_model)

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x


## 5. POSITIONAL ENCODING

La **codificación posicional** se añade a los *embeddings* de entrada para proporcionar información sobre la posición de los tokens en la secuencia. A diferencia de las RNN y las LSTM, los transformers no captan de forma inherente la naturaleza secuencial de los datos, por lo que las codificaciones posicionales son esenciales para inyectar esta información.   

- *Codificación posicional*: Esta función crea una codificación única para cada posición de la secuencia, que se añade a los embeddings de token.
- *Seno* y *coseno*: Las posiciones se codifican utilizando funciones seno y coseno con diferentes frecuencias para distinguir las posiciones.

A continuación se muestra la función para calcular las codificaciones posicionales:

In [ ]:
# -----------------------------------------------------------------------------
# Clase 4: Positional Encoding
# -----------------------------------------------------------------------------
# El transformer no usa convoluciones ni recurencia, así que para
# que la red "entienda" la posición de cada token, se inyecta información
# posicional mediante senos y cosenos.
# -----------------------------------------------------------------------------
def positional_encoding(position, d_model):
    """
    Genera un tensor de tamaño (position, d_model) con el encoding posicional.
    """
    # Generamos las posiciones (0, 1, 2, ..., position-1)
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # Se aplican seno a las posiciones pares de la dimensión de modelo
    # y coseno a las posiciones impares
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])

    pos_encoding = np.zeros(angle_rads.shape)
    pos_encoding[:, 0::2] = sines
    pos_encoding[:, 1::2] = cosines

    # Devolvemos pos_encoding con una dimensión extra para batch
    return tf.cast(pos_encoding[np.newaxis, ...], dtype=tf.float32)

def get_angles(pos, i, d_model):
    """
    Función auxiliar para calcular los ángulos para el encoding posicional.
    """
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

## 6. CAPA DE ENCODER   

El codificador esta formado por varias capas codificadoras. Convierte la secuencia de entrada en un conjunto de 'embeddings' enriquecidos con información posicional.   

- `Encoder`: El codificador se compone de una capa de 'embedding', una de codificación posicional, una de dropout y múltiples bloques `transformer`. Procesa la secuencia de entrada y genera una representación de la secuencia.
- `call`: La secuencia de entrada pasa por la capa de 'embedding', se añade la codificación posicional y, a continuación, atraviesa secuencialmente los bloques `transformer`.

In [ ]:


# -----------------------------------------------------------------------------
# Clase 5: Capa de Encoder
# -----------------------------------------------------------------------------
# Cada capa del encoder consiste en:
#   1. Multi-head attention (con residual connection + layer normalization).
#   2. Feed Forward (de dos capas, con residual connection + layer normalization).
# -----------------------------------------------------------------------------
class EncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedForward(d_model, ff_dim)

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, x, mask=None, training=False):
        # Multi-head attention
        attn_output, _ = self.mha(x, x, x, mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        # Feed Forward
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

## 7. CLASE ENCODER

In [ ]:




# -----------------------------------------------------------------------------
# Clase 6: Encoder
# -----------------------------------------------------------------------------
# El encoder está compuesto por:
#   - Embedding + Positional Encoding
#   - N capas de EncoderLayer
# -----------------------------------------------------------------------------
class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, ff_dim,
                 input_vocab_size, maximum_position_encoding, dropout_rate=0.1, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_layers = num_layers

        # Capa de embedding (para tokens)
        self.embedding = layers.Embedding(input_vocab_size, d_model)

        # Cálculo de la codificación posicional
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.enc_layers = [
            EncoderLayer(d_model, num_heads, ff_dim, dropout_rate)
            for _ in range(num_layers)
        ]
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x, mask=None, training=False):
        seq_len = tf.shape(x)[1]

        # Sumar embedding y codificación posicional
        x = self.embedding(x)  # (batch_size, seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, mask=mask, training=training)

        return x

## 8. CAPA DE DECODER   

El descodificador genera la secuencia de salida a partir de la representación codificada utilizando mecanismos para atender tanto a la salida del codificador como a los tokens generados previamente.   

- `call`:La secuencia de entrada pasa por el 'embedding' y la codificación posicional y, a continuación, por los bloques `transformer` del descodificador.

In [ ]:

# -----------------------------------------------------------------------------
# Clase 7: Capa de Decoder
# -----------------------------------------------------------------------------
# Cada capa del decoder consiste en:
#   1. Masked multi-head attention (para que el decoder no vea "futuro").
#   2. Multi-head attention recibiendo la salida del encoder.
#   3. Feed Forward.
# Cada bloque con sus conexiones residuales y normalización de capa.
# -----------------------------------------------------------------------------
class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(DecoderLayer, self).__init__(**kwargs)
        self.mha1 = MultiHeadAttention(d_model, num_heads)  # masked MHA
        self.mha2 = MultiHeadAttention(d_model, num_heads)  # MHA con salida del encoder

        self.ffn = PositionwiseFeedForward(d_model, ff_dim)

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.dropout3 = layers.Dropout(dropout_rate)

    def call(self, x, enc_output, look_ahead_mask=None, padding_mask=None, training=False):
        # 1) Masked multi-head attention
        attn1, attn_weights_block1 = self.mha1(x, x, x, mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(x + attn1)

        # 2) Multi-head attention con la salida del encoder
        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        # 3) Feed Forward
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(out2 + ffn_output)

        return out3, attn_weights_block1, attn_weights_block2

## 9. CLASE DECODER

In [ ]:
# -----------------------------------------------------------------------------
# Clase 8: Decoder
# -----------------------------------------------------------------------------
# El decoder está compuesto por:
#   - Embedding + Positional Encoding
#   - N capas de DecoderLayer
# -----------------------------------------------------------------------------
class Decoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, ff_dim,
                 target_vocab_size, maximum_position_encoding, dropout_rate=0.1, **kwargs):
        super(Decoder, self).__init__(**kwargs)

        self.d_model = d_model
        self.num_layers = num_layers

        # Embedding para la secuencia de salida
        self.embedding = layers.Embedding(target_vocab_size, d_model)

        # Codificación posicional
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [
            DecoderLayer(d_model, num_heads, ff_dim, dropout_rate)
            for _ in range(num_layers)
        ]
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x, enc_output, look_ahead_mask=None, padding_mask=None, training=False):
        seq_len = tf.shape(x)[1]

        # Sumar embedding y codificación posicional
        x = self.embedding(x)  # (batch_size, seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        attention_weights = {}
        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x,
                                                   enc_output,
                                                   look_ahead_mask=look_ahead_mask,
                                                   padding_mask=padding_mask,
                                                   training=training)
            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        return x, attention_weights


## 10. TRANSFORMER COMPLETO (ENCODER+DECODER)   

El modelo final combina el codificador y el descodificador y genera las predicciones finales.

In [ ]:
# -----------------------------------------------------------------------------
# Clase 9: Transformer completo (Encoder + Decoder)
# -----------------------------------------------------------------------------
class Transformer(keras.Model):
    def __init__(self, num_layers, d_model, num_heads, ff_dim,
                 input_vocab_size, target_vocab_size, 
                 pe_input, pe_target, dropout_rate=0.1, **kwargs):
        super(Transformer, self).__init__(**kwargs)

        self.encoder = Encoder(num_layers, d_model, num_heads, ff_dim,
                               input_vocab_size, pe_input, dropout_rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, ff_dim,
                               target_vocab_size, pe_target, dropout_rate)

        self.final_layer = layers.Dense(target_vocab_size)

    def call(self, inputs, training=False, enc_padding_mask=None,
             look_ahead_mask=None, dec_padding_mask=None):
        """
        inputs: tupla (inp, tar)
            inp -> secuencia de entrada (batch, seq_len_in)
            tar -> secuencia de salida (batch, seq_len_out)
        """

        inp, tar = inputs
        # Salida del encoder
        enc_output = self.encoder(inp, mask=enc_padding_mask, training=training)

        # Salida del decoder
        dec_output, attention_weights = self.decoder(
            tar, enc_output,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask,
            training=training
        )
        
        # Guardamos los pesos de atención en un atributo, 
        # pero no los retornamos como salida "oficial" del modelo
        self._attention_weights = attention_weights

        # Capa final densa para predecir el siguiente token
        final_output = self.final_layer(dec_output)  # (batch_size, seq_len_out, vocab_size)

        return final_output
    
    def get_attention_weights(self):
        """Método auxiliar para acceder a los pesos de atención si se desea."""
        return self._attention_weights
    

## 11. EJEMPLO DE EJECUCIÓN

In [ ]:

# -----------------------------------------------------------------------------
# EJEMPLO DE USO
# -----------------------------------------------------------------------------
# A continuación construimos un pequeño Transformer para mostrar cómo se usa.
# No se entrena con datos reales, simplemente ilustra la compilación y llamada.

if __name__ == "__main__":
    # Parámetros
    num_layers = 2       # número de capas en encoder/decoder
    d_model = 128        # dimensión de embeddings
    num_heads = 4        # número de cabezas en Multi-Head Attention
    ff_dim = 512         # dimensión interna de la red feed-forward
    input_vocab_size = 8500
    target_vocab_size = 8000
    dropout_rate = 0.1

    # Creamos un Transformer
    transformer = Transformer(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        ff_dim=ff_dim,
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        pe_input=10000,   # máximo de posiciones en la entrada
        pe_target=6000,   # máximo de posiciones en la salida
        dropout_rate=dropout_rate
    )

    # Compilamos el modelo con optimizador y pérdida
    # (Por ejemplo, la entropía cruzada categórica si se tratara de un problema de clasificación de tokens)
    transformer.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    # Generamos datos de ejemplo (batch_size=2, secuencia de 5 tokens)
    dummy_inp = tf.constant([[1,2,3,4,5],[4,5,6,7,0]])
    dummy_tar = tf.constant([[1,2,3,4,5],[2,3,4,5,6]])

    # Llamamos al modelo
    # normalmentte deberíamos construir máscaras, pero para un ejemplo sencillo las omitimos
    pred, attn = transformer((dummy_inp, dummy_tar), training=False)

    print("Forma de la salida:", pred.shape)  # (batch_size, seq_len_out, target_vocab_size)
    # Por ejemplo -> (2, 5, 8000)

    # Probamos un paso de entrenamiento "dummy"
    # Para entrenamiento real, se necesitan datos reales y máscaras adecuadas
    history = transformer.fit(x=(dummy_inp, dummy_tar),
                              y=dummy_tar,  # normalmente: y son los mismos tar desplazados
                              epochs=1,
                              verbose=1)

### ***LOGITS***
Los *logits* son las salidas sin normalizar de la red. Se suelen usar en combinación con la función de pérdida de entropía cruzada, que internamente aplica la operación necesaria (softmax + cálculo de entropía cruzada) de la forma más eficiente y estable.   
Cuando una red neuronal realiza una clasificación (imaginemos que queremos clasificar entre varias clases), la última capa suele producir un vector de dimensión igual al número de clases. A este vector se le llaman “logits”.

## RESULTADO DE EJECUCIÓN DEL TRANSFORMER
**Forma de la salida: (5, 8000)**:

El modelo está produciendo un tensor cuyas dimensiones son (sequence_length, vocab_size).
En este caso, la secuencia de salida tiene 5 posiciones (tokens) y el vocabulario tiene 8000 posibles tokens diferentes.

Si se dispone de un batch size de 1 (una sola secuencia de entrada/salida en este ejemplo), a veces TensorFlow "colapsa" la dimensión de batch y acaba mostrándo solo (5, 8000).
Si se tiene un batch_size mayor a 1, típicamente se vería algo como (batch_size, seq_length, vocab_size).

Indica que se está ejecutando una sola iteración de entrenamiento (un minibatch) durante la epoch (puesto que solo se ha proporcionado un conjunto de datos muy pequeño, de tamaño 1).


**La precisión (accuracy) es 0.0.**

Esto es normal en un solo paso de entrenamiento con datos aleatorios y pesos aleatorios. Si el modelo no acierta ninguno de los tokens, la exactitud para ese batch es 0.

**loss: 9.0551:**

Es el valor de la función de pérdida (en este caso, entropía cruzada).
Un valor alrededor de 9 es típico cuando se genera una secuencia de logits muy dispersos sobre 8000 posibles tokens, sin haber realizado un entrenamiento real.

En resumen, el mensaje de salida está indicando:

- Qué forma tiene la salida de el Transformer (la capa final produce logits de dimensión [seq_len, vocab_size] o [batch_size, seq_len, vocab_size], si el batch_size no se colapsa).

- Qué ocurrió en ese único paso de entrenamiento de ejemplo: la exactitud fue 0 y la pérdida aproximadamente 9.05, algo esperado al no haber entrenado con datos reales ni por varias épocas.